<a href="https://www.kaggle.com/code/klimyuriy/beer-reviews-predict?scriptVersionId=192659070" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection, metrics
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
import joblib
import optuna
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/beerreviews/beer_reviews.csv


In [11]:
df=pd.read_csv('/kaggle/input/beerreviews/beer_reviews.csv')
df

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586609,14359,The Defiant Brewing Company,1162684892,5.0,4.0,3.5,maddogruss,Pumpkin Ale,4.0,4.0,The Horseman's Ale,5.2,33061
1586610,14359,The Defiant Brewing Company,1161048566,4.0,5.0,2.5,yelterdow,Pumpkin Ale,2.0,4.0,The Horseman's Ale,5.2,33061
1586611,14359,The Defiant Brewing Company,1160702513,4.5,3.5,3.0,TongoRad,Pumpkin Ale,3.5,4.0,The Horseman's Ale,5.2,33061
1586612,14359,The Defiant Brewing Company,1160023044,4.0,4.5,4.5,dherling,Pumpkin Ale,4.5,4.5,The Horseman's Ale,5.2,33061


In [12]:
filtered_beer_reviews_df = df.drop(columns=['brewery_id', 'review_time', 'review_profilename', 'beer_beerid', 'beer_name'])

In [13]:
filtered_beer_reviews_df.describe()

,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06
mean,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00
std,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00
min,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02
25%,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00
50%,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00
75%,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00
max,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01


In [14]:
filtered_beer_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   brewery_name       1586599 non-null  object 
 1   review_overall     1586614 non-null  float64
 2   review_aroma       1586614 non-null  float64
 3   review_appearance  1586614 non-null  float64
 4   beer_style         1586614 non-null  object 
 5   review_palate      1586614 non-null  float64
 6   review_taste       1586614 non-null  float64
 7   beer_abv           1518829 non-null  float64
dtypes: float64(6), object(2)
memory usage: 96.8+ MB


In [15]:
#filtered_beer_reviews_df['beer_abv'].fillna(0, inplace=True)
filtered_beer_reviews_df.dropna(subset=['beer_abv'], inplace=True)
filtered_beer_reviews_df.dropna(subset=['brewery_name'], inplace=True)
#filtered_beer_reviews_df.replace(0, 0.1, inplace=True)
filtered_beer_reviews_df

,brewery_name,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv
0,Vecchio Birraio,1.5,2.0,2.5,Hefeweizen,1.5,1.5,5.0
1,Vecchio Birraio,3.0,2.5,3.0,English Strong Ale,3.0,3.0,6.2
2,Vecchio Birraio,3.0,2.5,3.0,Foreign / Export Stout,3.0,3.0,6.5
3,Vecchio Birraio,3.0,3.0,3.5,German Pilsener,2.5,3.0,5.0
4,Caldera Brewing Company,4.0,4.5,4.0,American Double / Imperial IPA,4.0,4.5,7.7
...,...,...,...,...,...,...,...,...
1586609,The Defiant Brewing Company,5.0,4.0,3.5,Pumpkin Ale,4.0,4.0,5.2
1586610,The Defiant Brewing Company,4.0,5.0,2.5,Pumpkin Ale,2.0,4.0,5.2
1586611,The Defiant Brewing Company,4.5,3.5,3.0,Pumpkin Ale,3.5,4.0,5.2
1586612,The Defiant Brewing Company,4.0,4.5,4.5,Pumpkin Ale,4.5,4.5,5.2


In [51]:
columns_to_check = ['review_aroma', 'review_appearance', 'review_palate', 'review_taste', "beer_abv"]

# Фільтруємо рядки, де в будь-якій з зазначених колонок є значення 0
rows_with_zeros = filtered_beer_reviews_df[(filtered_beer_reviews_df[columns_to_check] == 0).any(axis=1)]

# Виведемо знайдені рядки
rows_with_zeros

,brewery_name,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv,total_score,w_aroma,w_appearance,w_palate,w_taste,weighted_average


In [50]:
# Фільтрація рядків, де в будь-якій з зазначених колонок є значення 0
filtered_beer_reviews_df = filtered_beer_reviews_df[~(filtered_beer_reviews_df[columns_to_check] == 0).any(axis=1)]

# Виведення кількості рядків після фільтрації
filtered_beer_reviews_df.shape

(1518807, 14)

In [18]:
# Фільтрація рядків, де у всіх зазначених колонках значення дорівнює 0
rows_with_all_zeros = filtered_beer_reviews_df[(filtered_beer_reviews_df[columns_to_check] == 0).all(axis=1)]

# Виведення знайдених рядків
rows_with_all_zeros

,brewery_name,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv


In [20]:
rows_with_overall_zero = filtered_beer_reviews_df[filtered_beer_reviews_df['review_overall'] == 0]

# Виведення знайдених рядків
rows_with_overall_zero

,brewery_name,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv
37234,Desert Edge Brewery,0.0,2.0,0.0,Czech Pilsener,3.0,2.0,4.0
39648,Desert Edge Brewery,0.0,4.0,0.0,Irish Dry Stout,2.0,2.0,4.0
39667,Desert Edge Brewery,0.0,3.0,0.0,American Pale Ale (APA),2.0,2.0,4.0
653695,Red Rock Brewing Company,0.0,2.0,0.0,Hefeweizen,2.5,2.0,4.0
653706,Red Rock Brewing Company,0.0,3.0,0.0,American Amber / Red Ale,2.5,3.5,4.0
653726,Red Rock Brewing Company,0.0,2.0,0.0,German Pilsener,3.0,1.5,4.0
653740,Red Rock Brewing Company,0.0,2.0,0.0,Dunkelweizen,2.5,2.0,4.0


In [46]:
rows_with_appearance_and_overall_zero = filtered_beer_reviews_df[(filtered_beer_reviews_df['review_appearance'] == 0) & (filtered_beer_reviews_df['review_overall'] == 0)]

# Виведення знайдених рядків
rows_with_appearance_and_overall_zero

,brewery_name,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv,total_score,w_aroma,w_appearance,w_palate,w_taste,weighted_average
37234,Desert Edge Brewery,0.0,2.0,0.0,Czech Pilsener,3.0,2.0,4.0,7.0,0.285714,0.0,0.428571,0.285714,2.428571
39648,Desert Edge Brewery,0.0,4.0,0.0,Irish Dry Stout,2.0,2.0,4.0,8.0,0.500000,0.0,0.250000,0.250000,3.000000
39667,Desert Edge Brewery,0.0,3.0,0.0,American Pale Ale (APA),2.0,2.0,4.0,7.0,0.428571,0.0,0.285714,0.285714,2.428571
653695,Red Rock Brewing Company,0.0,2.0,0.0,Hefeweizen,2.5,2.0,4.0,6.5,0.307692,0.0,0.384615,0.307692,2.192308
653706,Red Rock Brewing Company,0.0,3.0,0.0,American Amber / Red Ale,2.5,3.5,4.0,9.0,0.333333,0.0,0.277778,0.388889,3.055556
653726,Red Rock Brewing Company,0.0,2.0,0.0,German Pilsener,3.0,1.5,4.0,6.5,0.307692,0.0,0.461538,0.230769,2.346154
653740,Red Rock Brewing Company,0.0,2.0,0.0,Dunkelweizen,2.5,2.0,4.0,6.5,0.307692,0.0,0.384615,0.307692,2.192308


In [29]:
rows_with_aroma_zero = filtered_beer_reviews_df[filtered_beer_reviews_df['review_aroma'] == 0]

# Фільтрація рядків, де значення у стовпці review_palate дорівнює 0
rows_with_palate_zero = filtered_beer_reviews_df[filtered_beer_reviews_df['review_palate'] == 0]

# Фільтрація рядків, де значення у стовпці review_taste дорівнює 0
rows_with_taste_zero = filtered_beer_reviews_df[filtered_beer_reviews_df['review_taste'] == 0]

In [30]:
rows_with_aroma_zero

,brewery_name,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv


In [31]:
rows_with_palate_zero

,brewery_name,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv


In [32]:
rows_with_taste_zero.count()

brewery_name         0
review_overall       0
review_aroma         0
review_appearance    0
beer_style           0
review_palate        0
review_taste         0
beer_abv             0
dtype: int64

In [44]:
rows_with_same_appearance_and_overall = filtered_beer_reviews_df[filtered_beer_reviews_df['review_palate'] == filtered_beer_reviews_df['review_overall']]

rows_with_same_appearance_and_overall

,brewery_name,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv,total_score,w_aroma,w_appearance,w_palate,w_taste,weighted_average
0,Vecchio Birraio,1.5,2.0,2.5,Hefeweizen,1.5,1.5,5.0,7.5,0.266667,0.333333,0.200000,0.200000,1.966667
1,Vecchio Birraio,3.0,2.5,3.0,English Strong Ale,3.0,3.0,6.2,11.5,0.217391,0.260870,0.260870,0.260870,2.891304
2,Vecchio Birraio,3.0,2.5,3.0,Foreign / Export Stout,3.0,3.0,6.5,11.5,0.217391,0.260870,0.260870,0.260870,2.891304
4,Caldera Brewing Company,4.0,4.5,4.0,American Double / Imperial IPA,4.0,4.5,7.7,17.0,0.264706,0.235294,0.235294,0.264706,4.264706
5,Caldera Brewing Company,3.0,3.5,3.5,Herbed / Spiced Beer,3.0,3.5,4.7,13.5,0.259259,0.259259,0.222222,0.259259,3.388889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586603,The Defiant Brewing Company,4.0,4.5,4.0,Pumpkin Ale,4.0,4.0,5.2,16.5,0.272727,0.242424,0.242424,0.242424,4.136364
1586605,The Defiant Brewing Company,5.0,5.0,5.0,Pumpkin Ale,5.0,5.0,5.2,20.0,0.250000,0.250000,0.250000,0.250000,5.000000
1586606,The Defiant Brewing Company,5.0,4.5,4.0,Pumpkin Ale,5.0,5.0,5.2,18.5,0.243243,0.216216,0.270270,0.270270,4.662162
1586607,The Defiant Brewing Company,5.0,5.0,4.5,Pumpkin Ale,5.0,4.5,5.2,19.0,0.263158,0.236842,0.263158,0.236842,4.763158


In [38]:
# Розрахунок відносних ваг
filtered_beer_reviews_df['total_score'] = filtered_beer_reviews_df['review_aroma'] + filtered_beer_reviews_df['review_appearance'] + filtered_beer_reviews_df['review_palate'] + filtered_beer_reviews_df['review_taste']

filtered_beer_reviews_df['w_aroma'] = filtered_beer_reviews_df['review_aroma'] / filtered_beer_reviews_df['total_score']
filtered_beer_reviews_df['w_appearance'] = filtered_beer_reviews_df['review_appearance'] / filtered_beer_reviews_df['total_score']
filtered_beer_reviews_df['w_palate'] = filtered_beer_reviews_df['review_palate'] / filtered_beer_reviews_df['total_score']
filtered_beer_reviews_df['w_taste'] = filtered_beer_reviews_df['review_taste'] / filtered_beer_reviews_df['total_score']

# Виведення результатів
filtered_beer_reviews_df[['w_aroma', 'w_appearance', 'w_palate', 'w_taste']]


,w_aroma,w_appearance,w_palate,w_taste
0,0.266667,0.333333,0.200000,0.200000
1,0.217391,0.260870,0.260870,0.260870
2,0.217391,0.260870,0.260870,0.260870
3,0.250000,0.291667,0.208333,0.250000
4,0.264706,0.235294,0.235294,0.264706
...,...,...,...,...
1586609,0.258065,0.225806,0.258065,0.258065
1586610,0.370370,0.185185,0.148148,0.296296
1586611,0.250000,0.214286,0.250000,0.285714
1586612,0.250000,0.250000,0.250000,0.250000


In [39]:
filtered_beer_reviews_df[['w_aroma', 'w_appearance', 'w_palate', 'w_taste']].describe()

,w_aroma,w_appearance,w_palate,w_taste
count,1.518814e+06,1.518814e+06,1.518814e+06,1.518814e+06
mean,2.468660e-01,2.555201e-01,2.475367e-01,2.500772e-01
std,2.615150e-02,3.076296e-02,2.542096e-02,2.519662e-02
min,6.250000e-02,0.000000e+00,6.896552e-02,6.666667e-02
25%,2.333333e-01,2.368421e-01,2.333333e-01,2.400000e-01
50%,2.500000e-01,2.500000e-01,2.500000e-01,2.500000e-01
75%,2.608696e-01,2.692308e-01,2.608696e-01,2.647059e-01
max,6.250000e-01,6.250000e-01,5.384615e-01,5.714286e-01


In [42]:
filtered_beer_reviews_df['weighted_average'] = (
    filtered_beer_reviews_df['w_aroma'] * filtered_beer_reviews_df['review_aroma'] +
    filtered_beer_reviews_df['w_appearance'] * filtered_beer_reviews_df['review_appearance'] +
    filtered_beer_reviews_df['w_palate'] * filtered_beer_reviews_df['review_palate'] +
    filtered_beer_reviews_df['w_taste'] * filtered_beer_reviews_df['review_taste']
)

# Розрахунок середньої зваженої оцінки
mean_weighted_average = filtered_beer_reviews_df['weighted_average'].mean()

# Виведення середньої зваженої оцінки
mean_weighted_average

3.827135791497486

In [43]:
filtered_beer_reviews_df

,brewery_name,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv,total_score,w_aroma,w_appearance,w_palate,w_taste,weighted_average
0,Vecchio Birraio,1.5,2.0,2.5,Hefeweizen,1.5,1.5,5.0,7.5,0.266667,0.333333,0.200000,0.200000,1.966667
1,Vecchio Birraio,3.0,2.5,3.0,English Strong Ale,3.0,3.0,6.2,11.5,0.217391,0.260870,0.260870,0.260870,2.891304
2,Vecchio Birraio,3.0,2.5,3.0,Foreign / Export Stout,3.0,3.0,6.5,11.5,0.217391,0.260870,0.260870,0.260870,2.891304
3,Vecchio Birraio,3.0,3.0,3.5,German Pilsener,2.5,3.0,5.0,12.0,0.250000,0.291667,0.208333,0.250000,3.041667
4,Caldera Brewing Company,4.0,4.5,4.0,American Double / Imperial IPA,4.0,4.5,7.7,17.0,0.264706,0.235294,0.235294,0.264706,4.264706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586609,The Defiant Brewing Company,5.0,4.0,3.5,Pumpkin Ale,4.0,4.0,5.2,15.5,0.258065,0.225806,0.258065,0.258065,3.887097
1586610,The Defiant Brewing Company,4.0,5.0,2.5,Pumpkin Ale,2.0,4.0,5.2,13.5,0.370370,0.185185,0.148148,0.296296,3.796296
1586611,The Defiant Brewing Company,4.5,3.5,3.0,Pumpkin Ale,3.5,4.0,5.2,14.0,0.250000,0.214286,0.250000,0.285714,3.535714
1586612,The Defiant Brewing Company,4.0,4.5,4.5,Pumpkin Ale,4.5,4.5,5.2,18.0,0.250000,0.250000,0.250000,0.250000,4.500000


In [ ]:
X = filtered_beer_reviews_df.drop(columns=['review_overall'])
y = filtered_beer_reviews_df['review_overall']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  train_size=0.75,
                                                  random_state=42)

print(f'Train: {X_train.shape[0]} samples \t {round(X_train.shape[0] / X.shape[0] * 100)}%')
print(f'Test:  {X_test.shape[0]} samples \t {round(X_test.shape[0] / X.shape[0] * 100)}%')
print(f'Val:   {X_val.shape[0]} samples \t {round(X_val.shape[0] / X.shape[0] * 100)}%')

In [ ]:
num_columns = X.select_dtypes(include="number").columns
cat_columns = X.select_dtypes(include="object").columns

num_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("poly", PolynomialFeatures(degree=2, include_bias=False))
    ]
)
cat_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value =-1)),
          ("poly", PolynomialFeatures(degree=2, include_bias=False))])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns),
    ]
)

preprocessor.set_output(transform='pandas')
preprocessor

In [ ]:
X_train = preprocessor.fit_transform(X_train)

X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

In [ ]:
import lightgbm as lgb
from sklearn import metrics

def objective(trial):
    params = {
        'objective': trial.suggest_categorical('objective', ['poisson', 'mse', 'gamma']),
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'goss', 'rf']),
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'max_depth': trial.suggest_int('max_depth', -1, 200),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 200),
        'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf', 1e-3, 10.0),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 1.0),
        'max_bin': trial.suggest_int('max_bin', 15, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1e-3, 10.0),
        'is_unbalance': trial.suggest_categorical('is_unbalance', [True, False]),
        'metric': trial.suggest_categorical('l1', 'rmse'),
        'verbosity': -1,
        'force_col_wise': trial.suggest_categorical('force_col_wise', [True, False]),
        'max_cat_threshold': trial.suggest_int('max_cat_threshold', 1, 128),
        'cat_l2': trial.suggest_float('cat_l2', 1e-3, 10.0),
        'cat_smooth': trial.suggest_float('cat_smooth', 1e-3, 10.0),
        'max_cat_to_onehot': trial.suggest_int('max_cat_to_onehot', 1, 128),
        'top_rate': trial.suggest_float('top_rate', 0.0, 1.0),
        'other_rate': trial.suggest_float('other_rate', 0.0, 1.0 - trial.suggest_float('top_rate', 0.0, 1.0)),
        'min_data_per_group': trial.suggest_int('min_data_per_group', 1, 100),
        'max_cat_trees': trial.suggest_int('max_cat_trees', 1, 128),
        'path_smooth': trial.suggest_float('path_smooth', 0.0, 1.0),
        'feature_fraction_bynode': trial.suggest_float('feature_fraction_bynode', 0.0, 1.0),
        'tweedie_variance_power': trial.suggest_float('tweedie_variance_power', 1.0, 2.0),
    }

    model = lgb.LGBMRegressor(**params)

    early_stopping_callback = lgb.early_stopping(stopping_rounds=trial.suggest_int('early_stopping_round', 10, 100), verbose=False)

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[early_stopping_callback]
    )

    y_pred = model.predict(X_val)
    r2 = metrics.r2_score(y_val, y_pred)
    mae = metrics.mean_absolute_error(y_val, y_pred)
    rmse = metrics.mean_squared_error(y_val, y_pred) ** 0.5

    return r2, mae, rmse

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(directions=['maximize', 'minimize', 'minimize'])
study.optimize(objective, n_trials=100)

In [ ]:
len(study.best_trials)

In [ ]:
print(study.best_trials)

In [ ]:
data = pd.DataFrame([trial.values for trial in study.best_trials],
                    columns=['r2', 'mae', 'rmse'])

data

In [ ]:
best_params = study.best_trials[0].params
best_params

In [ ]:
model = lgb.LGBMRegressor(**best_params,
                          n_estimator = 1000,
                          verbose=-1)

model.fit(X_train, y_train,
          eval_set=[(X_test, y_test), (X_train, y_train)],
          eval_metric='mae')

In [ ]:
def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

In [ ]:
all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, model.predict(X_test))

all_metrics

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
lgb.plot_importance(model, figsize=(7,5))

In [ ]:
lgb.plot_metric(model,
                metric='l1')

In [ ]:
model = Pipeline(
    [("prep", preprocessor),
     ("model", model)
     ]
)

model

In [ ]:
joblib.dump(model, 'Beer_reviews.pkl')

In [ ]:
import shutil

shutil.move('Beer_reviews.pkl', '/kaggle/working/Beer_reviews.pkl')

In [ ]:
load_model = joblib.load('Beer_reviews.pkl')
load_model